# NLP Spacy
*  https://realpython.com/natural-language-processing-spacy-python/

# Instalando bibliotecas

In [1]:
!pip install spacy -q
!pip install python-dotenv -q
!pip install pymongo -q
!pip install pydub -q
!python -m spacy info
!python -m spacy download pt_core_news_lg -q
!pip install SpeechRecognition -q
!pip install speechmatics-python -q


============================== Info about spaCy ==============================

spaCy version    3.7.6                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-6.1.85+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        pt_core_news_lg (3.7.0), en_core_web_sm (3.7.1)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 1.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [20]:
!pip uninstall tenacity -y

Found existing installation: tenacity 8.2.3
Uninstalling tenacity-8.2.3:
  Successfully uninstalled tenacity-8.2.3


In [21]:
!pip install tenacity~=8.2.3

  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
Using cached tenacity-8.2.3-py3-none-any.whl (24 kB)


# Importando bibliotecas

In [2]:
import nltk
nltk.download('punkt')
import spacy
from spacy import displacy
from pymongo import MongoClient
import pandas as pd
import numpy as np
from spacy.language import Language
from spacy.tokenizer import Tokenizer
from collections import Counter
from spacy.matcher import Matcher
import os
from dotenv import load_dotenv, dotenv_values
from pydub import AudioSegment
import librosa
from google.colab import files
from speech_recognition import Recognizer, AudioFile
from speechmatics.models import ConnectionSettings
from speechmatics.batch_client import BatchClient
from httpx import HTTPStatusError

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Carregando variável de ambiente

In [3]:
load_dotenv()
# print(os.getenv("conn_str"))

True

# Conectando a base do MongoDB

In [4]:
conn_str = os.getenv("conn_str")

try:
    client = MongoClient(conn_str)
    print("Conexão sucedida ao MongoDB!")
except Exception as e:
    print(f"Erro: {e}")

Conexão sucedida ao MongoDB!


# Acessando a base e coleção

In [5]:
db = client["Cluster"]
collection = db["Audios_gold"]

db.list_collection_names()

['Audios_gold']

# Identificando pelo nome do arquivo

In [6]:
transcricao = collection.find({}, {"transcription": 1, "_id": 0, "file_name": 1})

transcricoes = list(transcricao)

df = pd.DataFrame(transcricoes)

df.to_csv('transcricoes.csv', index=False)

In [7]:
df['transcription'] = df['transcription'].str.lower()
df

,file_name,transcription
0,2967602.wav,"alô, boa tarde. boa tarde. fala com o senhor c..."
1,2967344.wav,plastico lojas br. boa tarde! boa tarde. meu n...
2,2967631.wav,"eu tenho o bourbon. lorena, boa tarde. boa tar..."
3,2965935.wav,"alô, alô, bom dia. leandro eu novamente. tudo ..."
4,2965979.wav,edson paulista. joão paulo. bom dia. bom dia! ...
...,...,...
75,2967729.wav,boa tarde! boa tarde lilia da empresa totvs eu...
76,2966177.wav,"alô, carneiro, bom dia! bom dia beatriz da fot..."
77,2964530.wav,"alô, alô, alô, boa tarde! boa tarde, azevedo. ..."
78,2966819.wav,"hotel ibis em são carlos. silvana, bom dia! bo..."


In [8]:
df.to_csv('transcricoes.csv', index=False)

# Duração de cada áudio

In [9]:
pasta = "/content/drive/MyDrive/Challenge/Amostra de Dados"

audios = [audio for audio in os.listdir(pasta) if audio.endswith((".wav", ".mp3"))]

dados = []

for audio in audios:
    caminho_audio = os.path.join(pasta, audio)
    try:

        audio_segment = AudioSegment.from_file(caminho_audio)


        duracao_ms = len(audio_segment)


        minutos = duracao_ms // (1000 * 60)
        segundos = (duracao_ms % (1000 * 60)) // 1000
        milisegundos = duracao_ms % 1000


        if segundos >= 60:
            minutos += segundos // 60
            segundos = segundos % 60


        dados.append({
            'Nome do Arquivo': audio,
            'Minutos': minutos,
            'Segundos': segundos,
            'Milisegundos': milisegundos
        })
    except Exception as e:
        print(f"Erro ao processar {audio}: {e}")

df_audio = pd.DataFrame(dados)

df_audio['Tempo'] = df_audio.apply(lambda row: f"{row['Minutos']:02d}:{row['Segundos']:02d}:{row['Milisegundos']:03d}", axis=1)

df_audio = df_audio.drop(['Minutos', 'Segundos', 'Milisegundos'], axis=1)

df_audio = df_audio[['Nome do Arquivo', 'Tempo']]

In [10]:
df_audio

,Nome do Arquivo,Tempo
0,2964235.wav,03:29:550
1,2964530.wav,03:29:980
2,2962625.wav,01:33:450
3,2874774.wav,05:00:420
4,2962074.wav,11:02:320
...,...,...
75,2968253.wav,02:37:940
76,2964398.wav,03:02:040
77,2966651.wav,02:46:940
78,2965979.wav,03:08:970


In [11]:
df_audio.to_csv('audios.csv', index=False)

* Um botão OU um input no prompt/terminal para carregar pelo menos UM arquivo de áudio

In [12]:
carregado = files.upload()

Saving 2874774.wav to 2874774.wav


In [13]:
# audio = list(carregado.keys())[0]

* Impressão do texto contido no áudio

In [14]:
audio = AudioSegment.from_file("2874774.wav")
audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)
audio.export("2874774_converted.wav", format="wav")
recognizer = Recognizer()

with AudioFile("2874774_converted.wav") as source:
    audio_data = recognizer.record(source)
    text = recognizer.recognize_google(audio_data, language='pt-BR')

In [15]:
API_KEY = "009PUEwm7Y0kG8tYOJeWsZjfWAIr80Np"
PATH_TO_FILE = "/content/2874774.wav"
LANGUAGE = "pt"

settings = ConnectionSettings(
    url="https://asr.api.speechmatics.com/v2",
    auth_token=API_KEY,
)

conf = {
    "type": "transcription",
    "transcription_config": {
        "language": LANGUAGE,
        "enable_entities": True,
        "operating_point": "enhanced",
    },
}


with BatchClient(settings) as client:
    try:
        job_id = client.submit_job(
            audio=PATH_TO_FILE,
            transcription_config=conf,
        )
        print(f'job {job_id} submitted successfully, waiting for transcript')
        transcript = client.wait_for_completion(job_id, transcription_format='txt')
        print(transcript)
    except HTTPStatusError as e:
        if e.response.status_code == 401:
            print('Invalid API key - Check your API_KEY at the top of the code!')
        elif e.response.status_code == 400:
            print(e.response.json()['detail'])
        else:
            raise e

job tw1imjbqu6 submitted successfully, waiting for transcript
Escandinávia. Nathália, bom dia! Bom dia Natália. Beatriz falando Gostaria de falar com a senhora Silvana do TI. Seria ela ainda responsável pelo sistema? Só um momento. Obrigada. Silvana, bom dia. A senhora Silvana Beatriz da Toto, falando Tudo bem? Bom dia, Beatriz! Tudo bom em você? Estou bem, Obrigada. Silvana, A senhora seria a responsável pelo sistema na empresa Escandinávia Veículos, né? Sim, correto? Tá, Meu contato é bem rápido. Eu falo da parte da experiência do cliente. Eu gostaria de saber somente como que tá aí a experiência, acompanhar um pouco a percepção que vocês estão tendo. Esse acompanhamento é feito pelo método de passe, onde a senhora avalia de 0A10A marca em alguns pontos específicos. Dois minutinhos, Tudo bem. A senhora seria responsável pelo TI né? Sim. Uma das. Tá. Eu vou só confirmar o CNPJ porque vocês são empresas coligadas, né? Então uma rede, o CNPJ que eu tô falando é O670411116000 contra 21. 

* A) QUANTIDADE total de palavras

In [16]:
def contar_palavras(texto):
    """
    Conta a quantidade total de palavras em um texto.

    Args:
        texto: O texto a ser analisado.

    Returns:
        O número total de palavras no texto.
    """
    palavras = texto.split()
    return len(palavras)

# Exemplo de uso com o texto transcrito:
quantidade_palavras = contar_palavras(transcript)
print("O texto tem", quantidade_palavras, "palavras.")

O texto tem 639 palavras.


* B) QUANTIDADE de palavras por minuto

In [17]:
# Calcula a duração do áudio em minutos
duracao_audio_segundos = len(audio) / 1000
duracao_audio_minutos = duracao_audio_segundos / 60

# Calcula a quantidade de palavras por minuto
palavras_por_minuto = quantidade_palavras / duracao_audio_minutos

print("Quantidade de palavras por minuto:", palavras_por_minuto)
print("Duração do áudio em minutos:", duracao_audio_minutos)

Quantidade de palavras por minuto: 127.62133013780705
Duração do áudio em minutos: 5.007000000000001


In [18]:
81.83241973375097 * 2.5540000000000003

209.0

* C) Palavras listadas da mais falada para a menos falada, bem como a impressão da QUANTIDADE de vezes em que cada palavra aparece no texto

In [19]:
# Divide o texto em palavras
palavras = text.lower().split()

# Conta a frequência de cada palavra
contagem_palavras = Counter(palavras)

# Encontra a palavra mais falada
palavra_mais_falada = contagem_palavras.most_common(1)[0]

# Encontra a palavra menos falada
palavra_menos_falada = contagem_palavras.most_common()[:-2:-1][0]

# Encontra todas as palavras com a frequência da palavra mais falada
palavras_mais_faladas = [palavra for palavra, contagem in contagem_palavras.items() if contagem == palavra_mais_falada[1]]

# Encontra todas as palavras com a frequência da palavra menos falada
palavras_menos_faladas = [palavra for palavra, contagem in contagem_palavras.items() if contagem == palavra_menos_falada[1]]

print("Palavra(s) mais falada(s):", palavras_mais_faladas, " - Quantidade:", palavra_mais_falada[1])
print("Palavra(s) menos falada(s):", palavras_menos_faladas, " - Quantidade:", palavra_menos_falada[1])

Palavra(s) mais falada(s): ['a']  - Quantidade: 29
Palavra(s) menos falada(s): ['candidatos', 'silvânia', 'tire', 'ainda', 'momento', 'sara', 'escandinávia', 'veículos', 'meu', 'rápido', 'falo', 'parte', 'referência', 'cliente', 'saber', 'somente', 'acompanhar', 'percepção', 'estão', 'tendo', 'feito', 'método', 'inquérito', 'trabalhadeira', 'marca', 'falta', 'quantos', '2', 'minutinhos', 'poderia', 'confirmar', 'porque', 'empresas', 'ficou', 'ligadas', '67041', '1116', 'mil', 'contra', '21', 'ok', 'e-mail', 'continua', 'silvania.com.br', 'essa', 'filial', 'uberlândia', 'depois', '17', '4,60', 'vendo', 'aqui', 'corresponde', 'pelos', 'três', 'iniciar', 'aparecendo', 'me', 'fala', 'nada', 'dela', 'probabilidade', 'recomendar', 'todos', 'amigo', 'colega', 'nossa', 'pra', 'aprofundar', 'melhoria', 'agora', 'alguns', 'pontos', 'sapato', 'técnico', 'avalia', 'agilidade', 'nos', 'processos', 'chamados', 'abre', 'agente', 'analistas', 'lidam', 'vamos', 'comercial', 'executivo', 'vendas', 'valo

* D) PERCENTUAL de vezes que as “palavras mágicas” aparecem no texto: Obrigado(a), Por favor, Desculpa, Desculpe, Por gentileza, Bom dia, Boa tarde, Boa noite, Agradeço, Agradece, Gratidão, Sinto muito, Perdão, me perdoe, com licença.

In [20]:
# Lista de palavras mágicas
palavras_magicas = ["obrigado", "obrigada", "por favor", "desculpa", "desculpe",
                     "por gentileza", "bom dia", "boa tarde", "boa noite",
                     "agradeço", "agradece", "gratidão", "sinto muito",
                     "perdão", "me perdoe", "com licença"]

# Conta a frequência de cada palavra mágica no texto
contagem_palavras_magicas = {palavra: text.lower().count(palavra) for palavra in palavras_magicas}

# Calcula o número total de palavras mágicas
total_palavras_magicas = sum(contagem_palavras_magicas.values())

# Calcula o percentual de palavras mágicas no texto
percentual_palavras_magicas = (total_palavras_magicas / quantidade_palavras) * 100

# Imprime o percentual
print("Percentual de palavras mágicas no texto: {:.2f}%".format(percentual_palavras_magicas))

# Imprime a frequência de cada palavra mágica
for palavra, contagem in contagem_palavras_magicas.items():
    print("- {}: {} vezes".format(palavra, contagem))

Percentual de palavras mágicas no texto: 1.10%
- obrigado: 0 vezes
- obrigada: 3 vezes
- por favor: 0 vezes
- desculpa: 0 vezes
- desculpe: 0 vezes
- por gentileza: 0 vezes
- bom dia: 3 vezes
- boa tarde: 0 vezes
- boa noite: 0 vezes
- agradeço: 1 vezes
- agradece: 0 vezes
- gratidão: 0 vezes
- sinto muito: 0 vezes
- perdão: 0 vezes
- me perdoe: 0 vezes
- com licença: 0 vezes


* Normalização dos áudios

In [21]:
df['transcription'].str.len().sum()

219713

In [22]:
def separa_palavras(lista_tokens):
    lista_palavras = []
    for token in lista_tokens:
      if token.isalpha():
        lista_palavras.append(token)
    return lista_palavras

In [23]:
lista_tokens = nltk.tokenize.word_tokenize(df['transcription'].str.cat(sep=' '))
lista_palavras = separa_palavras(lista_tokens)
print(len(lista_palavras))

38129


In [24]:
print(lista_palavras[:5])

['alô', 'boa', 'tarde', 'boa', 'tarde']


In [25]:
def normalizacao(lista_palavras):
    lista_normalizada = []
    for palavra in lista_palavras:
      lista_normalizada.append(palavra.lower())
    return lista_normalizada

lista_normalizada = normalizacao(lista_palavras)
print(lista_normalizada[:5])

['alô', 'boa', 'tarde', 'boa', 'tarde']


In [26]:
len(set(lista_normalizada))

2642

In [27]:
# lista_normalizada

In [28]:
nlp = spacy.load("pt_core_news_lg")
corpus = " ".join(lista_normalizada)
doc = nlp(corpus)
type(doc)

spacy.tokens.doc.Doc

* Separar token/palavra

In [29]:
[token.text for token in doc]

['alô',
 'boa',
 'tarde',
 'boa',
 'tarde',
 'fala',
 'com',
 'o',
 'senhor',
 'clóvis',
 'gentili',
 'sim',
 'clóvis',
 'me',
 'chamo',
 'beatriz',
 'fala',
 'pra',
 'todos',
 'tudo',
 'bem',
 'tudo',
 'bem',
 'que',
 'bom',
 'você',
 'gostaria',
 'de',
 'falar',
 'com',
 'o',
 'senhor',
 'o',
 'senhor',
 'seria',
 'o',
 'responsável',
 'da',
 'cgc',
 'atacadista',
 'né',
 'é',
 'bem',
 'rápido',
 'meu',
 'contato',
 'senhor',
 'clóvis',
 'eu',
 'preciso',
 'só',
 'fazer',
 'um',
 'acompanhamento',
 'pra',
 'saber',
 'como',
 'tá',
 'a',
 'experiência',
 'e',
 'a',
 'satisfação',
 'de',
 'vocês',
 'esse',
 'acompanhamento',
 'dura',
 'dois',
 'minutinhos',
 'no',
 'máximo',
 'é',
 'feito',
 'pelo',
 'método',
 'de',
 'nps',
 'tudo',
 'bem',
 'tudo',
 'bem',
 'tá',
 'o',
 'senhor',
 'vai',
 'avaliar',
 'somente',
 'de',
 'marca',
 'totem',
 'alguns',
 'pontos',
 'que',
 'eu',
 'vou',
 'perguntar',
 'tá',
 'vou',
 'confirmar',
 'o',
 'cnpj',
 'cnpj',
 'oito',
 'três',
 'zero',
 'contra'

* Frequência de palavras

In [30]:
texto_completo = (corpus)
doc_completo = nlp(texto_completo)
palavras = [
    token.text
    for token in doc_completo
    if not token.is_stop and not token.is_punct and not token.like_num
]
print(Counter(palavras).most_common(10))

[('tá', 572), ('senhor', 335), ('gente', 260), ('certo', 247), ('contato', 229), ('senhora', 212), ('totvs', 203), ('nota', 203), ('falar', 189), ('atendimento', 185)]


* Menos frequentes

In [31]:
palavras = [
    token.text
    for token in doc_completo
    if not token.is_stop and not token.is_punct and not token.like_num
]
print(Counter(palavras).most_common()[:-6:-1])

[('limeira', 1), ('confiabilidade', 1), ('itá', 1), ('oliveira', 1), ('furtado', 1)]


* Mais frequentes

In [32]:
contador_plavras = Counter(
    [token.text for token in doc_completo if not token.is_punct]
)
print(contador_plavras.most_common(5))

[('a', 1605), ('o', 1123), ('de', 999), ('que', 916), ('não', 797)]


* Contador de palavras

In [33]:
def contar_palavras(texto):
    """
    Conta a quantidade total de palavras em um texto.

    Args:
        texto: O texto a ser analisado.

    Returns:
        O número total de palavras no texto.
    """
    palavras = texto.split()
    return len(palavras)

quantidade_palavras = contar_palavras(corpus)
print("O texto tem", quantidade_palavras, "palavras.")

O texto tem 38129 palavras.


* Lista de palavras mágicas

In [34]:
# Lista de palavras mágicas
palavras_magicas = ["obrigado", "obrigada", "por favor", "desculpa", "desculpe",
                     "por gentileza", "bom dia", "boa tarde", "boa noite",
                     "agradeço", "agradece", "gratidão", "sinto muito",
                     "perdão", "me perdoe", "com licença"]

# Conta a frequência de cada palavra mágica no texto
contagem_palavras_magicas = {palavra: (corpus).lower().count(palavra) for palavra in palavras_magicas}

# Calcula o número total de palavras mágicas
total_palavras_magicas = sum(contagem_palavras_magicas.values())

# Calcula o percentual de palavras mágicas no texto
percentual_palavras_magicas = (total_palavras_magicas / quantidade_palavras) * 100

# Imprime o percentual
print("Percentual de palavras mágicas no texto: {:.2f}%".format(percentual_palavras_magicas))

# Imprime a frequência de cada palavra mágica
for palavra, contagem in contagem_palavras_magicas.items():
    print("- {}: {} vezes".format(palavra, contagem))

Percentual de palavras mágicas no texto: 1.22%
- obrigado: 35 vezes
- obrigada: 155 vezes
- por favor: 29 vezes
- desculpa: 12 vezes
- desculpe: 5 vezes
- por gentileza: 4 vezes
- bom dia: 101 vezes
- boa tarde: 100 vezes
- boa noite: 0 vezes
- agradeço: 19 vezes
- agradece: 5 vezes
- gratidão: 0 vezes
- sinto muito: 0 vezes
- perdão: 1 vezes
- me perdoe: 1 vezes
- com licença: 0 vezes


* Separar quantidade de frases em um arquivo

In [35]:
doc_frases = nlp(corpus)
frases = list(doc_frases.sents)
len(frases)

185

In [36]:
print(
    f"{'Text with Whitespace':22}"
    f"{'Is Alphanumeric?':15}"
    f"{'Is Punctuation?':18}"
    f"{'Is Stop Word?'}"
)
for token in doc_frases:
    print(
        f"{str(token.text_with_ws):22}"
        f"{str(token.is_alpha):15}"
        f"{str(token.is_punct):18}"
        f"{str(token.is_stop)}"
    )

A saída de streaming foi truncada nas últimas 5000 linhas.
nota                  True           False             False
para                  True           False             True
as                    True           False             True
atualizações          True           False             False
uma                   True           False             True
nota                  True           False             False
para                  True           False             True
as                    True           False             True
atualizações          True           False             False
nove                  True           False             True
nove                  True           False             True
a                     True           False             True
última                True           False             False
pergunta              True           False             False
para                  True           False             True
avaliar               True         

* Demonstração de remoção de stop words

In [37]:
custom_doc_frases = (corpus)
nlp = spacy.load("pt_core_news_lg")
sobre_doc = nlp(custom_doc_frases)
print([token for token in sobre_doc if not token.is_stop and not token.is_punct])

[alô, fala, senhor, clóvis, gentili, clóvis, chamo, beatriz, fala, pra, gostaria, falar, senhor, senhor, responsável, cgc, atacadista, né, rápido, contato, senhor, clóvis, preciso, acompanhamento, pra, tá, experiência, satisfação, acompanhamento, dura, minutinhos, feito, método, nps, tá, senhor, avaliar, marca, totem, vou, perguntar, tá, vou, confirmar, cnpj, cnpj, confirmação, senhor, clóvis, escala, probabilidade, senhor, recomendar, totvs, amigo, colega, hoje, algum, comentário, senhor, gostaria, deixar, nota, gente, aprofundar, melhoria, vou, falar, específicos, peço, senhor, avalie, caso, tenha, contato, tá, falando, suporte, técnico, senhor, avalia, agilidade, processos, suporte, hoje, demorado, tá, falando, atendimento, agentes, suporte, né, falando, comercial, atendimento, executivo, vendas, falando, custos, valores, produtos, contratados, atendimento, financeiro, referente, boletos, negociações, caso, precisem, precisamos, colocar, hoje, satisfação, implantação, software, impl

* Diferentes categorizações de palavras


In [38]:
texto = (corpus)

doc_texto = nlp(texto)

for token in doc_texto:
    if str(token) != str(token.lemma_):
        print(f"{str(token):>20} : {str(token.lemma_)}")

A saída de streaming foi truncada nas últimas 5000 linhas.
            corrigiu : corrigir
               tenta : tentar
                  dá : dar
                  dá : dar
                  me : eu
                  dá : dar
                 uma : um
                fala : falar
                   a : o
            consegue : conseguir
               disso : de isso
               disso : de isso
               disso : de isso
             entendi : entender
                  dá : dar
                  dá : dar
                 uma : um
           assertiva : assertivo
          definitiva : definitivo
                   é : ser
               tiver : ter
                 uma : um
                 uma : um
          definitiva : definitivo
                  me : eu
             explica : explicar
                 tem : ter
             everton : Everton
                  tá : tar
               meses : mês
                   a : o
             retorna : retornar
                   a

In [39]:
texto_completo = (corpus)

completo_doc = nlp(texto_completo)

def toker_permitido(token):
    return bool(
        token
        and str(token).strip()
        and not token.is_stop
        and not token.is_punct
    )

def processar_token(token):
    return token.lemma_.strip().lower()

tokens_filtrados = [
    toker_permitido(token)
    for token in completo_doc
    if toker_permitido(token)
]

In [40]:
# tokens_filtrados

* Removendo stop words
* Removendo pontuações


In [41]:
texto_completo = (corpus)

doc_completo = nlp(texto_completo)

def token_permitido(token):
    return bool(
        token
        and str(token).strip()
        and not token.is_stop
        and not token.is_punct
    )

def processa_token(token):
    return token.lemma_.strip().lower()

tokens_filtrados_completos = [
    processa_token(token)
    for token in doc_completo
    if token_permitido(token)
]

In [42]:
tokens_filtrados_completos

['alô',
 'falar',
 'senhor',
 'clóvis',
 'gentili',
 'clóvis',
 'chamar',
 'beatriz',
 'falar',
 'pra',
 'gostar',
 'falar',
 'senhor',
 'senhor',
 'responsável',
 'cgc',
 'atacadista',
 'né',
 'rápido',
 'contato',
 'senhor',
 'clóvis',
 'preciso',
 'acompanhamento',
 'pra',
 'tá',
 'experiência',
 'satisfação',
 'acompanhamento',
 'durar',
 'minutinho',
 'fazer',
 'método',
 'nps',
 'tar',
 'senhor',
 'avaliar',
 'marca',
 'totem',
 'ir',
 'perguntar',
 'tar',
 'ir',
 'confirmar',
 'cnpj',
 'cnpj',
 'confirmação',
 'senhor',
 'clóvis',
 'escala',
 'probabilidade',
 'senhor',
 'recomendar',
 'totvs',
 'amigo',
 'colega',
 'hoje',
 'algum',
 'comentário',
 'senhor',
 'gostar',
 'deixar',
 'nota',
 'gente',
 'aprofundar',
 'melhoria',
 'ir',
 'falar',
 'específico',
 'peçer',
 'senhor',
 'avaliar',
 'caso',
 'ter',
 'contato',
 'tar',
 'falar',
 'suporte',
 'técnico',
 'senhor',
 'avaliar',
 'agilidade',
 'processo',
 'suporte',
 'hoje',
 'demorar',
 'tar',
 'falar',
 'atendimento',
 'a

* Classificação da palavra

In [43]:
adjetivos = [token.text for token in doc_completo if token.pos_ == "ADJ"]
adjetivos_unicos = list(set(adjetivos))
adjetivos_unicos = [token for token in nlp(' '.join(adjetivos_unicos)) if token.pos_ == "ADJ"]
print(adjetivos_unicos)

substantivos = [token.text for token in doc_completo if token.pos_ == "NOUN"]
substantivos_unicos = list(set(substantivos))
substantivos_unicos = [token for token in nlp(' '.join(substantivos_unicos)) if token.pos_ == "NOUN"]
print(substantivos_unicos)

adverbios = [token.text for token in doc_completo if token.pos_ == "ADV"]
adverbios_unicos = list(set(adverbios))
adverbios_unicos = [token for token in nlp(' '.join(adverbios_unicos)) if token.pos_ == "ADV"]
print(adverbios_unicos)

pontuacao = [token.text for token in doc_completo if token.pos_ == "PUNCT"]
pontuacao_unicos = list(set(pontuacao))
print(pontuacao_unicos)

verbos = [token.text for token in doc_completo if token.pos_ == "VERB"]
verbos_unicos = list(set(verbos))
verbos_unicos = [token for token in nlp(' '.join(verbos_unicos)) if token.pos_ == "VERB"]
print(verbos_unicos)

numeros = [token.text for token in doc_completo if token.pos_ == "NUM"]
numeros_unicos = list(set(numeros))
numeros_unicos = [token for token in nlp(' '.join(numeros_unicos)) if token.pos_ == "NUM"]
print(numeros_unicos)

pronomes = [token.text for token in doc_completo if token.pos_ == "PRON"]
pronomes_unicos = list(set(pronomes))
pronomes_unicos = [token for token in nlp(' '.join(pronomes_unicos)) if token.pos_ == "PRON"]
print(pronomes_unicos)

[especiais, curto, única, segunda, único, ponciano, gratuita, social, necessário, possível, lógico, diferente, específica, operacional, contínuas, natalia, quentinho, comentário, verde, necessária, feliz, pontuais, civil, ótima, próxima, leiga, prestativas, lento, simples, brasileira, maiores, enorme, horripilante, prime, pesquisada, quartas, ruim, menores, geral, importante, personal, complexo, melhores, capaz, legal, difícil, claro, atento, hein, válido, atacadista, responsável, preciso, complicado, adequado, obrigada, comum, gravíssimo, totas, navona, responsáveis, please, principais, industrial, ótimos, completo, grosso, totos, linda, complexas, inteiro, ridículo, seguinte, pequena, exclusivo, crocante, microbiológicos, bom, flexíveis, top, exorcista, inicial, digital, última, geovani, concorrente, pronto, quadrados, gerais, davin, mirrorless, último, específicos, piazza, boa, cartos, orgulhosos, final, escandinávia, christofoletti, menor, antiga, atencioso, lisa, rico, pequeno, es

In [44]:
client.close()

* Rodar uma nuvem de palavras para cada variável acima
* Fazer a normalização das palavras do dataframe para Lower e aplicar função lambda com os substantivos únicos para cada linha do Dataframe e outra coluna com os adjetivos únicos
* Aplicar análise de sentimento para essas colunas de adjetivo e substantivo com a biblioteca usada em SP3_ML(2) e avaliar o modelo

In [45]:
# transcricao = df['transcription']
# transcricao